In [11]:
from vexa import VexaAPI

In [12]:
vexa = VexaAPI()

In [13]:
meetings = vexa.get_meetings()

In [14]:
meetings[-1]

{'id': '47ce1725-63ff-41a8-9f20-1367e8298dd6',
 'title': 'call_2024-10-16T17:20:15+00:00',
 'start_timestamp': '2024-10-16T17:20:15Z',
 'finish_timestamp': None,
 'meeting': {'id': '3afe897f-c075-4ce5-a494-1b454b83a427',
  'external_id': 'zbv-dqxc-vxo',
  'title': None}}